In [4]:
import dataAck
import portfolio
from IPython.display import display
allTickers = dataAck.getAllTickersPlain()


In [ ]:
while True:
    import random
    tickerToTrade = allTickers[random.randint(0, len(allTickers)) - 1]
    print(tickerToTrade)

    tickersToPull = dataAck.getDataSourcesForTicker(tickerToTrade)
    print(tickersToPull)

    pulledData, validTickers = dataAck.downloadTickerData(tickersToPull)

    joinedData = dataAck.joinDatasets([pulledData[ticker] for ticker in pulledData])

    sManager = dataAck.seriesManager(validTickers)
    print(sManager.describe())

    import time
    import warnings
    import numpy as np
    warnings.filterwarnings("ignore")
    ##GET ALGOS INITIALLY GOOD
    runsSeen = 0
    while True:
        s = sManager.createSeries()
        while s.checkValidity(s.transformJoinedData(joinedData[:"2016-01-01"])) == False:
            s = sManager.createSeries()
            
        display(s.transformJoinedData(joinedData[:"2016-01-01"]))

        try:
            for defaultWindowSize in [5, 10, 22, 44]:
                for trees in [25, 50, 100, 150, 300]:
                    for predictionLength in [2, 3, 5, 7]:
                        if random.uniform(0,1) < 0.5: ##RANDOMLY SKIP
                            continue
                        b = dataAck.algoBlob(s, defaultWindowSize, trees, predictionLength, tickerToTrade)
                        algoReturn, factorReturn, predictions =  b.makePredictions(portfolio.prepareDataForModel(b, joinedData), daysToCheck = None, earlyStop = True)
                        if algoReturn is None:
                            toLog = factorReturn
                            toLog["modelDescription"] = str(b.describe())
                            dataAck.logModel("Model Stopped Early", toLog)
                            print("Model Stopped Early", toLog)
                            continue
                        metrics = dataAck.vizResults(algoReturn[:-252], factorReturn[:-252], False)
                        print("TRAIN:", metrics)
                        if np.isnan(metrics["SHARPE"]) == True:
                            raise ValueError('SHARPE IS NAN SO FAULTY SERIES')
                        if metrics["SHARPE"] > 1.0 and metrics["ACTIVITY"] > 0.7 and metrics["BETA"] < 0.3:
                            ##STORE
                            testMetrics = dataAck.vizResults(algoReturn[-252:], factorReturn[-252:], False)
                            print("TEST:", testMetrics)
                            print("TODAY:", b.makeTodayPrediction(portfolio.prepareDataForModel(b, joinedData)))

                            dataAck.storeModel(b, metrics, testMetrics)
                        else:
                            dataAck.logModel("Model Skipped", {
                                "modelDescription":str(b.describe()),
                                "sharpe":metrics["SHARPE"], ##OVERLOADED IN FAIL
                                "activity":metrics["ACTIVITY"],
                                "beta":metrics["BETA"]
                            })
        except:
            print("FAILED", s.describe())
            dataAck.logModel("Series Failed", {
                "seriesDescription":str(s.describe())
            })
            time.sleep(10)

        runsSeen += 1

        if runsSeen > 10:
            ##START NEW TICKER
            dataAck.logModel("Search Update", {
                "message":"restarting search with different ticker",
                "currentTicker":tickerToTrade
            })
            break

        

ITB
['AGG', 'DIA', 'DVY', 'DXJ', 'EEM', 'EFA', 'EWC', 'EWG', 'EWH', 'EWJ', 'EWT', 'EWU', 'EWW', 'EWY', 'EWZ', 'FEZ', 'FXE', 'FXI', 'GDX', 'GLD', 'IAU', 'IBB', 'IEF', 'IJH', 'IJR', 'ILF', 'IVE', 'IVV', 'IVW', 'IWB', 'IWD', 'IWF', 'IWM', 'IWN', 'IWO', 'IWR', 'IYF', 'IYR', 'IYT', 'KBE', 'KRE', 'LQD', 'MDY', 'OEF', 'OIH', 'QLD', 'QQQ', 'RSP', 'SDS', 'SDY', 'SH', 'SHY', 'SLV', 'SMH', 'SOXX', 'SPY', 'SSO', 'TIP', 'TLT', 'USO', 'VB', 'VBR', 'VFH', 'VGK', 'VGT', 'VIG', 'VNQ', 'VO', 'VTI', 'VTV', 'VUG', 'VWO', 'XBI', 'XLB', 'XLE', 'XLF', 'XLI', 'XLK', 'XLP', 'XLU', 'XLV', 'XLY', 'XME', 'XOP', 'XRT', 'ITB']
DXJ
AGG
DVY
EEM
DIA
EFA
EWG
EWH
EWC
EWT
FXI
EWU
EWJ
FXE
FEZ
EWY
EWZ
EWW
GDX
IAU
GLD
IBB
IEF
IJH
IJR
ILF
IVE
IVV
IWB
IVW
IWF
IWM
IWN
IWO
IWD
IWR
IYF
IYT
IYR
KBE
KRE
LQD
QLD
OIH
OEF
MDY
SDS
SDY
RSP
SH
QQQ
SLV
SHY
SSO
SMH
SOXX
TIP
USO
TLT
VB
VBR
SPY
VGK
VIG
VNQ
VGT
VO
VTV
VTI
VUG
VWO
XBI
XLE
XLB
XLI
XLF
XLK
XLP
XLV
XME
XLU
XLY
XOP
ITB
XRT
FAILED: VFH
APPLICABLE ETF ERROR: (<class 'quandl.errors.

Date
2006-09-11    5.717407
2006-09-12    2.151388
2006-09-13    4.892771
2006-09-14    4.655052
2006-09-15    1.412904
2006-09-18    1.110217
2006-09-19    0.945844
2006-09-20    1.041776
2006-09-21    1.121915
2006-09-22    1.099494
2006-09-25    1.040172
2006-09-26    0.963751
2006-09-27    1.022161
2006-09-28    1.185814
2006-09-29    1.160364
2006-10-02    1.109975
2006-10-03    1.047600
2006-10-04    0.812182
2006-10-05    0.626770
2006-10-06    0.748100
2006-10-09    0.652448
2006-10-10    0.169487
2006-10-11    0.023903
2006-10-12   -0.009795
2006-10-13   -0.393762
2006-10-16   -1.815772
2006-10-17   -2.402387
2006-10-18   -1.429296
2006-10-19   -1.450299
2006-10-20   -1.391466
                ...   
2015-11-18    0.907032
2015-11-19    1.212108
2015-11-20    1.136751
2015-11-23    0.976389
2015-11-24    0.756798
2015-11-25   -0.610941
2015-11-27    0.594333
2015-11-30    3.049402
2015-12-01    2.734002
2015-12-02    1.877500
2015-12-03    1.604627
2015-12-04    1.901798
2015-1

(5, ('XOP', 37, 17, None, 2), 2, 'ITB')
SERIES (5, ('XOP', 37, 17, None, 2), 2, 'ITB') WINDOW 5 TREES 25
Model Stopped Early {'sharpe': -0.8236624884763557, 'beta': 0.16554718072753619, 'activity': 0.5496031746031746, 'modelDescription': "((5, ('XOP', 37, 17, None, 2), 2, 'ITB'), 5, 25)"}
(5, ('XOP', 37, 17, None, 2), 5, 'ITB')
SERIES (5, ('XOP', 37, 17, None, 2), 5, 'ITB') WINDOW 5 TREES 25
Model Stopped Early {'sharpe': 0.29576608204941046, 'beta': 0.30959424694646326, 'activity': 0.9186507936507936, 'modelDescription': "((5, ('XOP', 37, 17, None, 2), 5, 'ITB'), 5, 25)"}
(5, ('XOP', 37, 17, None, 2), 7, 'ITB')
SERIES (5, ('XOP', 37, 17, None, 2), 7, 'ITB') WINDOW 5 TREES 25
Model Stopped Early {'sharpe': -0.57953855732802517, 'beta': 0.21880479741919615, 'activity': 0.9742063492063492, 'modelDescription': "((5, ('XOP', 37, 17, None, 2), 7, 'ITB'), 5, 25)"}
(5, ('XOP', 37, 17, None, 2), 5, 'ITB')
SERIES (5, ('XOP', 37, 17, None, 2), 5, 'ITB') WINDOW 5 TREES 100
Model Stopped Early {'s